In [2]:
import pandas as pd
import os
import numpy as np

In [123]:
PATH = "./wandb_classification_cnn_bp_0505.csv"
dataset = pd.read_csv(PATH)
PATH = "./wandb_classification_cnn_ccl_0505.csv"
dataset2 = pd.read_csv(PATH)
dataset = pd.concat([dataset, dataset2])

# del dataset["Created"], dataset["Runtime"], dataset["GradC"], dataset["act_B"], dataset["act_F"], dataset["epochs"], dataset["loss_scale_ssl"]
# del dataset["User"], dataset["Group"], dataset["Job Type"], dataset["Tags"], dataset["Sweep"], dataset["Description"], dataset["GPU Count"], dataset["best_epoch"], dataset["num_chn"], dataset["tmax"], dataset["warmup"], dataset["eta_min"]
# if one of the above colum is in the dataset, delete it
for col_name in dataset.columns:
    if col_name in ["Created", "Runtime", "GradC", "act_B", "act_F", "epochs", "loss_scale_ssl", "User", "Group", "Job Type", "Tags", "Sweep", "Description", "GPU Count", "best_epoch", "num_chn", "tmax", "warmup", "eta_min"]:
        del dataset[col_name]

print("Size of dataset: ", len(dataset))
dataset = dataset[dataset["State"]=="finished"]
dataset = dataset[dataset["architecture"]=="cnn_pool"]
# dataset = dataset[dataset["batchsize"]==64]
# dataset = dataset[dataset["fw_bn"]==0]
print("Size of dataset: ", len(dataset))

Size of dataset:  1328
Size of dataset:  1328


In [124]:
dataset[dataset.method=="CCL"].dataset.unique()

array(['CIFAR10', 'STL10_cls', 'CIFAR100'], dtype=object)

In [173]:
for task in ["CIFAR10", "CIFAR100", "STL10_cls"]:
    for method in ["BP", "CCL"]:
        ds = dataset[dataset["dataset"] == task]
        ds = ds[ds["method"] == method]
        if task == "CIFAR10" and method == "CCL":
            ds = ds[ds["lr_F"] != 1.2]
        ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).mean()
        ds_count = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).count()
        ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).std()
        ds_mean = ds_mean[ds_count["test_acc"] > 2]
        ds_std = ds_std[ds_count["test_acc"] > 2]
        try:
            row_with_highest_val_acc = ds_mean["val_acc"].idxmax()
        except:
            continue
        print("dataset: ", task, " method: ", method)
        print(f"mean: {ds_mean.loc[row_with_highest_val_acc]['test_acc']*100:.2f} std: {ds_std.loc[row_with_highest_val_acc]['test_acc']*100:.2f} count: {ds_count.loc[row_with_highest_val_acc]['test_acc']}")
        # if task == "CIFAR10" and method == "CCL":
        #     assert 0 == 1
        # if task == "STL10_cls" and method == "CCL":
        #     assert 0 == 1

dataset:  CIFAR10  method:  BP
mean: 87.12 std: 1.76 count: 10
dataset:  CIFAR10  method:  CCL
mean: 82.94 std: 0.53 count: 5
dataset:  CIFAR100  method:  BP
mean: 51.92 std: 0.48 count: 5
dataset:  CIFAR100  method:  CCL
mean: 56.29 std: 0.25 count: 5
dataset:  STL10_cls  method:  BP
mean: 51.27 std: 1.90 count: 5
dataset:  STL10_cls  method:  CCL
mean: 45.28 std: 2.58 count: 5


/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_5706/1683781218.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).mean()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_5706/1683781218.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).std()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_5706/1683781218.py:7: FutureWarning: The default

AssertionError: 